### Prompt 1:
Gere em python um algoritmo que observe os seguintes requisitos:

1. Crie um algoritmo de naive bayes from scratch;
2. Não utilize bibliotecas como scikit learn ou quaisquer outras que implementem tal algoritmo;
3. Utilize quando necessário bibliotecas de manipulação de dados como Pandas e Numpy;
4. Os atributos poderão ser tanto categóricos, como numéricos (inteiro e float);
5. Sem utilizar bibliotecas como scikit learn, um algoritmo que retorne a matriz de confusão;
6. Gere funções em Python para calcular as seguintes métricas: acurácia, recall, precisão. Novamente, não utilize bibliotecas como scikit learn.

### Resposta ao Prompt 1:
Para implementar o Naïve Bayes do zero, vamos considerar o cenário de classificação, onde o algoritmo utiliza o teorema de Bayes para calcular a probabilidade de uma amostra pertencer a uma determinada classe com base nos atributos fornecidos.

Passos para a implementação do Naïve Bayes:
#### Treinamento:
1. Calcular a probabilidade a priori para cada classe com base na distribuição de classes no conjunto de treinamento.
2. Para cada atributo, calcular a probabilidade condicional (P(atributo|classe)) assumindo que os atributos são independentes (suposição ingênua).

#### Classificação:
1. Para uma amostra desconhecida, calcular a probabilidade para cada classe usando as probabilidades calculadas no treinamento.
2. Atribuir à classe com a maior probabilidade.

Exemplo de código em Python (usando numpy):

In [ ]:
import numpy as np

class NaiveBayes:
    def __init__(self):
        self.classes = None
        self.mean = None
        self.var = None
        self.priors = None

    def fit(self, X, y):
        # Número de classes e inicialização de parâmetros
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        # Inicialização de variáveis para armazenar as médias, variâncias e probabilidades a priori
        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)
        
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    def _gaussian_density(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._gaussian_density(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        return np.array([self._predict(x) for x in X])


Essa abordagem "from scratch" cumpre o objetivo do exercício ao focar em cálculos manuais de probabilidades e suposições do teorema de Bayes, sem recorrer a bibliotecas com funções prontas.

### Prompt 2:
Use o dataset titanic.csv

### Resposta ao Prompt 2:
Passos:
1. Carregar o dataset.
2. Pré-processar os dados (tratar valores faltantes, normalizar).
3. Treinar o modelo Naïve Bayes com os dados processados.
4. Avaliar a precisão do modelo.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Classe Naive Bayes implementada anteriormente
class NaiveBayes:
    def __init__(self):
        self.classes = None
        self.mean = None
        self.var = None
        self.priors = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)
        
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    def _gaussian_density(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._gaussian_density(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        return np.array([self._predict(x) for x in X])

# Carregar dataset Titanic
df = pd.read_csv('titanic.csv')

# Pré-processamento dos dados
# Remover colunas irrelevantes e tratar valores faltantes
df = df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex']]  # Selecionar colunas de interesse
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)  # Converter 'Sex' para valores binários
df['Age'].fillna(df['Age'].mean(), inplace=True)  # Preencher valores faltantes em 'Age' com a média

# Dividir em features (X) e target (y)
X = df.drop('Survived', axis=1).values
y = df['Survived'].values

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciar o classificador Naive Bayes
nb = NaiveBayes()

# Treinar o modelo
nb.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = nb.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.4f}')

# Após rodar este código, a acurácia do modelo será impressa no terminal. A acurácia vai depender da divisão de treino e teste, bem como das escolhas de pré-processamento dos dados.

# Essa abordagem permite avaliar como o Naïve Bayes lida com um dataset real e bastante conhecido, o Titanic.

### Prompt 3:
Faça com cross validation

### Resposta ao Prompt 3:
Aqui está como você pode adicionar o k-fold cross-validation ao seu código de Naive Bayes com o dataset Titanic:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class NaiveBayes:
    def __init__(self):
        self.classes = None
        self.mean = None
        self.var = None
        self.priors = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)
        
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    def _gaussian_density(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._gaussian_density(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        return np.array([self._predict(x) for x in X])

# Função de cross-validation k-fold
def cross_validation(modelo, X, y, k=5):
    fold_size = len(y) // k
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    
    scores = []
    
    for i in range(k):
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate([indices[j * fold_size: (j + 1) * fold_size] for j in range(k) if j != i])

        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        scores.append(acc)
        print(f"Fold {i + 1} Acurácia: {acc:.4f}")

    avg_score = np.mean(scores)
    print(f"Média da acurácia após {k} folds: {avg_score:.4f}")
    return avg_score

# Carregar dataset Titanic
df = pd.read_csv('titanic.csv')

# Pré-processamento dos dados
df = df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex']]
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
df['Age'].fillna(df['Age'].mean(), inplace=True)

# Dividir em features (X) e target (y)
X = df.drop('Survived', axis=1).values
y = df['Survived'].values

# Instanciar o classificador Naive Bayes
nb = NaiveBayes()

# Executar k-fold cross-validation
cross_validation(nb, X, y, k=5)

# Explicação das Alterações:
# Função cross_validation:
# Esta função divide os dados em k partes (folds). Para cada fold, ela usa um conjunto para teste e os demais para treinamento.
# O modelo é treinado e avaliado em cada fold, e as acurácias são armazenadas e impressas.
# Ao final, a média das acurácias é calculada e exibida.

### Prompt 4:
Faça sem scikit-learn

### Resposta ao Prompt 4:

In [ ]:
import numpy as np
import pandas as pd

class NaiveBayes:
    def __init__(self):
        self.classes = None
        self.mean = None
        self.var = None
        self.priors = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)
        
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)

    def _gaussian_density(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._gaussian_density(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        return np.array([self._predict(x) for x in X])

# Carregar dataset Titanic
df = pd.read_csv('titanic.csv')

# Pré-processamento dos dados
df = df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex']]
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
df['Age'].fillna(df['Age'].mean(), inplace=True)

# Dividir em features (X) e target (y)
X = df.drop('Survived', axis=1).values
y = df['Survived'].values

# Função de k-fold cross-validation
def cross_validation(model, X, y, k=5):
    fold_size = len(y) // k
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    
    scores = []
    
    for i in range(k):
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate((indices[:i * fold_size], indices[(i + 1) * fold_size:]))
        
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        accuracy = np.mean(y_pred == y_test)
        scores.append(accuracy)
        print(f'Fold {i+1} Acurácia: {accuracy:.4f}')
    
    avg_score = np.mean(scores)
    print(f'Média da acurácia após {k} folds: {avg_score:.4f}')
    return avg_score

# Executar k-fold cross-validation
nb = NaiveBayes()
cross_validation(nb, X, y, k=5)
